<a href="https://colab.research.google.com/github/noelsj007/Pneumonia_detection_model/blob/main/PneumoniaModel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import random
import cv2
import pandas as pd

In [3]:
main_path = "/content/drive/MyDrive/data/pneumoniadataset"
# Define the paths to the dataset folders
train_dir = '/content/drive/MyDrive/data/pneumoniadataset/train'
valid_dir = '/content/drive/MyDrive/data/pneumoniadataset/val'
test_dir = '/content/drive/MyDrive/data/pneumoniadataset/test'

# Checking if the path exists
if os.path.exists(valid_dir):
    print("Path exists")
else:
    print("Path does not exist")

Path exists


In [4]:
def preprocess_data(directory, target_size, batch_size):
    """
    Preprocesses the data by applying normalization and augmentation.
    Data cleaning is not required as dataset is well prepared and properly labelled.

    Augmentation : to improve the model's performance , Normalization : model convergence
    Data augmentation techniques used : shear transformations, zoom transformations, and horizontal flipping

    Args:
        directory (str): Path to the dataset directory.
        target_size (tuple): Target size for resizing the images.
        batch_size (int): Batch size for loading the images.

    Returns:
        Preprocessed data generator.
    """
    # Normalize pixel values to [0, 1] for model convergence
    # Apply random shear & zoom transformations and Flip images horizontally

    data_gen = ImageDataGenerator(rescale=1./255,shear_range=0.3,zoom_range=0.3,horizontal_flip=False)
    data = data_gen.flow_from_directory(directory,target_size=target_size,batch_size=batch_size,class_mode='binary')
    return data

In [5]:
# Set the parameters for data preprocessing and augmentation
image_size = (224, 224)  # Reshape the images to this size
batch_size = 32

print("\npreprocessing of training data")
# Preprocess the training data
train_data = preprocess_data(train_dir, image_size, batch_size)

print("\npreprocessing of validation data")
# Preprocess the validation data
valid_data = preprocess_data(valid_dir, image_size, batch_size)

print("\npreprocessing of test data")
# Preprocess the test data
test_data = preprocess_data(test_dir, image_size, batch_size)


preprocessing of training data
Found 5227 images belonging to 2 classes.

preprocessing of validation data
Found 16 images belonging to 2 classes.

preprocessing of test data
Found 624 images belonging to 2 classes.


In [6]:
classnames = list(train_data.class_indices.keys())
print("classnames : ", classnames)

# Get the number of images in the train data
tr_count = train_data.n
print("Number of Images : ", tr_count)

#retrieving the image width, height and number of channels respectively
img_width, img_height, channels = train_data.image_shape
print(f"width : {img_width} \nheight :  {img_height} \nchannels : {channels}")

#copying
i_width,i_height,i_channels = img_width,img_height,channels

#retrieving number of classes
classes_count=train_data.num_classes
print("number of classes : ",classes_count)

class_labels = train_data.classes
print("Class labels:", class_labels)

#count number of samples in each class
class_counts = np.bincount(class_labels)
for class_name, num_samples in zip(classnames, class_counts):
    print(f"{class_name}: {num_samples}")

classnames :  ['NORMAL', 'PNEUMONIA']
Number of Images :  5227
width : 224 
height :  224 
channels : 3
number of classes :  2
Class labels: [0 0 0 ... 1 1 1]
NORMAL: 1342
PNEUMONIA: 3885


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping


# Define the CNN model

model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(classes_count, activation='softmax') #the number of output units based on the number of classes
])

# Compile the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)

# Training of the model with early stopping
model.fit(train_data, epochs=30, validation_data=valid_data, callbacks=[early_stopping])

# Training of the model
#model.fit(train_data, epochs=30, validation_data=valid_data)

Epoch 1/30
 69/164 [===========>..................] - ETA: 11:06 - loss: 0.5593 - accuracy: 0.7871